# EDA and simple model

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.linear_model import Ridge
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory
%matplotlib inline
from matplotlib import pyplot as plt
import seaborn as sns
from subprocess import check_output
import xgboost as xgb
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
df_train = pd.read_csv('../input/train.csv')
df_test = pd.read_csv('../input/test.csv')
df_test.head()

### Price

In [ ]:
low, high = df_train['price_doc'].quantile([0.01,0.99])
print (low,high)
df_train_1_99 = df_train[(df_train['price_doc']>=low) & (df_train['price_doc']<=high)]

## EDA

In [ ]:
df_train_price_mean = df_train['price_doc'].groupby(df_train['timestamp']).mean()
df_train_timestamp = df_train['timestamp'].value_counts().index
plt.figure()
plt.plot(range(len(df_train_timestamp)), df_train_price_mean)
plt.xlabel('timestamp')
plt.ylabel('avg_price')
plt.show()

In [ ]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
df_train_numeric = df_train_1_99.select_dtypes(include=numerics)
df_test_numeric = df_test.select_dtypes(include=numerics)

### Select some numeric features

In [ ]:
df_train_numeric_corr = df_train_numeric.corr()
#sns.heatmap(df_train_numeric_corr)

In [ ]:
df_train_numeric_corr_imp = df_train_numeric_corr['price_doc'].sort_values()
print (df_train_numeric_corr_imp)

In [ ]:
imp_columns = df_train_numeric_corr_imp[:10].index.append(df_train_numeric_corr_imp[:-10].index)
print(imp_columns)

### Simple XGB

In [ ]:
y_train = df_train_numeric['price_doc'].values.ravel()
X_train = df_train_numeric[imp_columns].values
X_test = df_test_numeric[imp_columns].values

In [ ]:
param={}
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)
model = xgb.train(param, dtrain)
d_test_result = model.predict(dtest)

In [ ]:
submission_df = df_test[['id']].copy()
submission_df.loc[:,'price_doc']=d_test_result
submission_df.head()

In [ ]:
submission_df.to_csv('output.csv',header=True,index=False)